In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

os.chdir('/content/drive/MyDrive/Chatbot/classifier_model/')

### Importing the libraries

In [3]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv("emotion-emotion_69k - emotion-emotion_69k.csv")

In [5]:
#print first 5 rows
df.head(2)

,Unnamed: 0,Situation,emotion,empathetic_dialogues,labels,Unnamed: 5,Unnamed: 6
0,0,I remember going to the fireworks with my best...,sentimental,Customer :I remember going to see the firework...,"Was this a friend you were in love with, or ju...",NaN,NaN
1,1,I remember going to the fireworks with my best...,sentimental,Customer :This was a best friend. I miss her.\...,Where has she gone?,NaN,NaN


In [6]:
#print the shape of the data set
print(df.shape)

(64636, 7)


In [7]:
df = df[['Situation', 'emotion']]

In [8]:
df.isnull().sum()

Situation    0
emotion      5
dtype: int64

In [9]:
df = df.dropna()

In [10]:
df.isnull().sum()

Situation    0
emotion      0
dtype: int64

In [11]:
df.duplicated().sum()

45333

In [12]:
df = df.drop_duplicates()

In [13]:
#check if the data is balanced or not
df.emotion.value_counts()

emotion
surprised       1000
excited          741
angry            686
proud            669
sad              663
annoyed          662
grateful         638
lonely           633
afraid           626
impressed        617
terrified        617
hopeful          616
guilty           613
disgusted        613
anxious          612
confident        611
furious          599
anticipating     598
joyful           598
nostalgic        596
disappointed     595
prepared         589
jealous          579
content          570
embarrassed      558
devastated       557
sentimental      515
caring           503
trusting         499
ashamed          490
apprehensive     463
faithful         372
Name: count, dtype: int64

### Text Preprocessing

In [14]:
df.head()

,Situation,emotion
0,I remember going to the fireworks with my best...,sentimental
5,i used to scare for darkness,afraid
10,I showed a guy how to run a good bead in weldi...,proud
14,I have always been loyal to my wife.,faithful
17,A recent job interview that I had made me feel...,terrified


In [15]:
df.reset_index(drop=True, inplace = True)

In [16]:
df.shape

(19298, 2)

In [17]:
df.head()

,Situation,emotion
0,I remember going to the fireworks with my best...,sentimental
1,i used to scare for darkness,afraid
2,I showed a guy how to run a good bead in weldi...,proud
3,I have always been loyal to my wife.,faithful
4,A recent job interview that I had made me feel...,terrified


In [18]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,WordNetLemmatizer
from nltk.corpus import stopwords

In [19]:
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from bs4 import BeautifulSoup

nltk.download('all')

stop_words = set(stopwords.words('english'))

def remove_html_tags(text):
    # Remove HTML tags using BeautifulSoup
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()

def remove_special_characters(text):
    # Remove special characters using regex
    return re.sub(r'[^a-zA-Z0-9\s]', '', text)

def remove_emojis(text):
    # Remove emojis using regex
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F700-\U0001F77F"  # alchemical symbols
                               u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                               u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                               u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                               u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                               u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                               u"\U00002702-\U000027B0"  # Dingbats
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def preprocess_text(text):

    # Remove HTML tags
    text = remove_html_tags(text)
    # Remove special characters
    text = remove_special_characters(text)
    # Remove emojis
    text = remove_emojis(text)
    # Convert to lowercase
    text = text.lower()
    text = re.sub(r'\d+', '', text)  # removes digits
    #remove punctuations
    words = text.translate(str.maketrans("", "", string.punctuation))
    # Tokenization
    words = word_tokenize(text)
    # Remove stopwords
    words = [word for word in words if word not in stop_words]
    # lemmatization for finding root words
    wordnet = WordNetLemmatizer()
    words = [wordnet.lemmatize(word) for word in words]
    # Join the words back into a sentence
    preprocessed_text = ' '.join(words)

    return preprocessed_text

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

In [20]:
# Apply text cleaning to the 'text' column
df['Situation'] = df['Situation'].apply(preprocess_text)

<ipython-input-19-b4541ee497be>:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


In [21]:
df['emotion'].unique()

array(['sentimental', 'afraid', 'proud', 'faithful', 'terrified',
       'joyful', 'angry', 'sad', 'jealous', 'grateful', 'prepared',
       'embarrassed', 'excited', 'annoyed', 'lonely', 'ashamed', 'guilty',
       'surprised', 'nostalgic', 'confident', 'furious', 'disappointed',
       'caring', 'trusting', 'disgusted', 'anticipating', 'anxious',
       'hopeful', 'content', 'impressed', 'apprehensive', 'devastated'],
      dtype=object)

In [22]:
df['emotion'].nunique()

32

In [23]:
emotion_replacements = {
    'surprised': 0,'excited': 1,'angry':2,'proud':3,'sad':4,'annoyed':5,'grateful':6,'lonely':7,'afraid':8,'hopeful':9,
    'terrified':10,'anxious':11,'guilty':12,'impressed':13,'disgusted':14,'confident':15,'anticipating':16,'joyful':17,
    'nostalgic':18,'furious':19,'disappointed':20,'prepared':21,'jealous':22,'content':23,'embarrassed':24,'devastated':25,
    'sentimental':26,'caring':27,'trusting':28,'ashamed':29,'apprehensive':30,'faithful':31}

# Replace emotions using the dictionary
df['emotion'] = df['emotion'].replace(emotion_replacements)

In [24]:
df['emotion'].unique()

array([26,  8,  3, 31, 10, 17,  2,  4, 22,  6, 21, 24,  1,  5,  7, 29, 12,
        0, 18, 15, 19, 20, 27, 28, 14, 16, 11,  9, 23, 13, 30, 25])

In [25]:
df['Situation'].head(10)

0    remember going firework best friend lot people...
1                                  used scare darkness
2    showed guy run good bead welding class caught ...
3                                    always loyal wife
4    recent job interview made feel anxious felt li...
5           happy first student year enginering school
6                                     lost job got mad
7         one year christmas get single gift rough day
8    work coworker called boss office allowed work ...
9    went park set bench didnt notice wallet felt m...
Name: Situation, dtype: object

In [26]:
x = df['Situation']
y = df['emotion']

In [27]:
# Check the indices
print(df.index)

RangeIndex(start=0, stop=19298, step=1)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
from imblearn.over_sampling import SMOTE,RandomOverSampler
resampled_X,resampled_Y=RandomOverSampler().fit_resample(vectorized_x, y)

In [ ]:
resampled_X

<32000x9445 sparse matrix of type '<class 'numpy.float64'>'
	with 262349 stored elements in Compressed Sparse Row format>

In [ ]:
resampled_Y.value_counts()

emotion
26    1000
8     1000
30    1000
13    1000
23    1000
9     1000
11    1000
16    1000
14    1000
28    1000
27    1000
20    1000
19    1000
15    1000
18    1000
0     1000
12    1000
29    1000
7     1000
5     1000
1     1000
24    1000
21    1000
6     1000
22    1000
4     1000
2     1000
17    1000
10    1000
31    1000
3     1000
25    1000
Name: count, dtype: int64

In [ ]:
# Train-test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(resampled_X, resampled_Y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB()

In [ ]:
classifier.fit(X_train, y_train)

MultinomialNB()

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
Accuracy = accuracy_score(y_test, y_pred)
print(Accuracy)

0.59046875


In [ ]:
Precision = precision_score(y_test, y_pred, average = 'weighted')
print(Precision)

0.5908129888230864


In [ ]:
Recall = recall_score(y_test, y_pred, average = 'micro')
print(Recall)

0.59046875


In [ ]:
F1 = f1_score(y_test, y_pred, average = 'macro')
print(F1)

0.5843211144397178
